Als erstes wird unsloth installiert, das ist ein Framework welches das Training vereinfacht und die Resourcenauslastung, durch optimierung der Mathematik, verbessert.

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [1]:
!pip install --no-deps --upgrade "flash-attn>=2.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.3-cp311-cp311-linux_x86_64.whl size=191363917 sha256=b1243e9b86687348a5ab03a073abacdf8e3d5e9e4b7e5326a183f47348c5dfba
  Stored in directory: /root/.cache/pip/wheels/d0/a3/f9/48d2706cb2eac05ec0dc144bf6954fe47bb3c2cd0de280765e
Successfully built flash-attn


Importieren der Bibliotheken und definieren der Kontextlänge und Quantisierung. Wir benutzen 4bit Datentypen um die RAM-Auslastung zu reduzieren zu lasten der Qualität des Modells. Hier könnte man auch 8bit oder 16bit benutzen, wenn genug Hardware vorhanden ist.

In [2]:
from unsloth import FastLanguageModel
import torch
from tqdm.notebook import tqdm
max_seq_length = 1024 #2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+

#prompt = 'You are an email classificator, please decide if the following mail is important for a night shift worker to be solved directly or if it could wait until business hours. Notifications that something is solved are unimportant. Imporant are only things that prevents from someone in a warehouse to work. respond only with important, unimportant'
prompt = 'you are an email classificator, classify the following email with important or unimportant'

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Als nächstes laden wir das Model Gemma 2 27B Instruct

In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-27b-it-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = True,
    load_in_8bit = False,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.1.6: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Jetzt werden die Mails aus der Excelliste eingelesen

In [4]:
import datasets
import pandas as pd

In [35]:
trainmails = pd.read_excel('Emails with expected classification.xlsx')
trainmails

,Subject,Body,Recipient,expected classification
0,Stoermeldung SM_WERK_SW_10.01.2025_02 aufgetreten,Fehlerklasse: 3\nPLT-SYSTEM: LE <> LE auf Lage...,dummy-inbox@organization.com,important
1,Neuer Kommentar zu SM_WERK_VO_14.01.2025_01,Name der Stoerungsmeldung: SM_WERK_VO_14.01.20...,dummy-inbox@organization.com,unimportant
2,Projektupdate: Fortschritt in Sprint 5,"Hallo Team,\n\nDas Projekt Sprint 5 hat einen ...",dummy-inbox@organization.com,unimportant
3,[External] Incident assigned to your group: IN...,An incident has been assigned to your group.\n...,dummy-inbox@organization.com,important
4,Projektupdate: Fortschritt in Sprint 5,"Hallo Team,\n\nDas Projekt Sprint 5 hat einen ...",dummy-inbox@organization.com,unimportant
...,...,...,...,...
487,Systembenachrichtigung: Fehler aufgetreten in ...,"Sehr geehrte Damen und Herren,\n\nEin Fehler w...",dummy-inbox@organization.com,important
488,Projektupdate: Fortschritt in Sprint 7,"Hallo Team,\n\nDas Projekt Sprint 7 hat einen ...",dummy-inbox@organization.com,unimportant
489,[TicketBCS#288646] Request to update alternati...,"Dear David,\n\nPlease update the alternative b...",dummy-inbox@organization.com,unimportant
490,Mitarbeiterfeedback benoetigt: Rueckmeldung bi...,"Hallo Team,\n\nWir moechten Sie daran erinnern...",dummy-inbox@organization.com,unimportant


Die Mailbodys werden jetzt in die Chat-Struktur für das LLama Model konvertiert. Dabei wird die richtige Antwort des Models schon vorgegeben, dass dieses trainiert werden kann.

In [7]:
texts = []
for _, row in tqdm(trainmails[:350].iterrows()):
  mailbody = row['Body'].strip()
  messages = [
      {"role": "user", "content": prompt + "\n\n"+ mailbody},
      {"role": "assistant", "content": row['expected classification']}
  ]
  inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=False, tokenize=False)
  texts.append(inputs)

dataset = datasets.Dataset.from_dict({'text':texts})
dataset[0]

0it [00:00, ?it/s]

{'text': '<bos><start_of_turn>user\nyou are an email classificator, classify the following email with important or unimportant\n\nFehlerklasse: 3\nPLT-SYSTEM: LE <> LE auf Lagerpunkt\nTransaktion: ZYM0529\nLaststand: 4412\nLE-Nummer: \nFA-Nummer: \nKefa-Nummer: \nKommiliste: \nWas soll getan werden: Ereignis loeschen\nWelche Loesung wird erwartet:\nWelche Fehler/Fehlermeldungen gab es: Siehe Screenshot, Problem ist schon mehrfach aufgetreten\nBetroffen: PLT-SYSTEM<end_of_turn>\n<start_of_turn>model\nimportant<end_of_turn>\n'}

Wir erstellen ein LoRa auf dem Model, also ein kleiner Patch mit welchen das große Model angepasst werden kann

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.6 patched 46 layers with 46 QKV layers, 46 O layers and 46 MLP layers.


Definieren der Trainingsparameter

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 0,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 120,
        learning_rate = 5e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/350 [00:00<?, ? examples/s]

Und los geht das Training

In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 350 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 120
 "-____-"     Number of trainable parameters = 114,180,096


Step,Training Loss
1,4.325900
2,3.918600
3,3.759700
4,3.233500
5,3.765300
6,3.479000
7,2.962900
8,3.093000
9,2.453100
10,2.243700


Speichern vom Model

In [19]:
model.save_pretrained("lora_model_llama")
tokenizer.save_pretrained("lora_model_llama")

('lora_model_llama/tokenizer_config.json',
 'lora_model_llama/special_tokens_map.json',
 'lora_model_llama/tokenizer.model',
 'lora_model_llama/added_tokens.json',
 'lora_model_llama/tokenizer.json')

Nach dem Training wird validiert, wir schalten in Interaktionsmodus

In [12]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 4608, padding_idx=0)
        (layers): ModuleList(
          (0-45): 46 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4608, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4608, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

Und führen das trainierte Model auf alle Mails aus, erstellen eine neue Spalte in Excel mit den Ausgaben und speichern das ergebnis als classified.xlsx

In [16]:
results = []
bodys = []
for _, row in tqdm(trainmails.iterrows()):
  mailbody = row.Body.strip()
  messages = [
      {"role": "user", "content": prompt + "\n\n" + mailbody},
  ]
  inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
  inputs = tokenizer(inputs, return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
  input_len = inputs['input_ids'].shape[-1]
  output_text = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True)

  if output_text.strip() != row['expected classification'].strip():
    print(f"expected {row['expected classification']}, classified: {output_text}")
    print(mailbody)
    print('-------')
  results.append(output_text)
  bodys.append(mailbody)
trainmails['category'] = results
trainmails.to_excel('classified.xlsx')


0it [00:00, ?it/s]

expected important, classified: unimportant

Fehlerklasse: 3
PLT-SYSTEM: LE <> LE auf Lagerpunkt
Transaktion: ZYM0529
Laststand: 4412
LE-Nummer: 
FA-Nummer: 
Kefa-Nummer: 
Kommiliste: 
Was soll getan werden: Ereignis loeschen
Welche Loesung wird erwartet:
Welche Fehler/Fehlermeldungen gab es: Siehe Screenshot, Problem ist schon mehrfach aufgetreten
Betroffen: PLT-SYSTEM
-------
expected important, classified: unimportant

Hallo Swati,

Folgende Tickets sind faellig zur Weiterbearbeitung:
#279794 NVN DMD Daily Monitoring

Hinweis: Dies ist eine automatisch generierte Mail!
Vielen Dank,
Service Desk
-------
expected important, classified: unimportant

Hallo Swati,

Folgende Tickets sind faellig zur Weiterbearbeitung:
#279794 NVN DMD Daily Monitoring

Hinweis: Dies ist eine automatisch generierte Mail!
Vielen Dank,
Service Desk
-------
expected important, classified: unimportant

Hallo Swati,

Folgende Tickets sind faellig zur Weiterbearbeitung:
#279794 NVN DMD Daily Monitoring

Hinweis: 

In [33]:
import ipywidgets as widgets
from IPython.display import display

textarea1 = widgets.Textarea(
    value='',
    placeholder='Enter mail text here...',
    description='Mail:',
    layout=widgets.Layout(width='100%', height='100px')
)

textarea2 = widgets.Textarea(
    value='',
    placeholder='response',
    description='Response:',
    layout=widgets.Layout(width='100%', height='100px')
)

# Create a button widget
button = widgets.Button(
    description='Classify',
    button_style='success'
)
button2 = widgets.Button(
    description='Add to Train',
    button_style='warning'
)

def on_button_add_train(b):
  i = len(trainmails)
  trainmails.loc[i, 'Body'] = textarea1.value.strip()
  trainmails.loc[i, 'expected classification'] = textarea2.value.strip()

# Define the button click event
def on_button_click(b):
    mailbody = textarea1.value.strip()
    messages = [
        {"role": "user", "content": prompt + "\n\n" + mailbody},
    ]
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
    inputs = tokenizer(inputs, return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    input_len = inputs['input_ids'].shape[-1]
    output_text = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True)

    textarea2.value = output_text

button.on_click(on_button_click)
button2.on_click(on_button_add_train)
# Display the widgets
display(textarea1, textarea2, button, button2)

Textarea(value='', description='Mail:', layout=Layout(height='100px', width='100%'), placeholder='Enter mail t…

Textarea(value='', description='Response:', layout=Layout(height='100px', width='100%'), placeholder='response…

Button(button_style='success', description='Classify', style=ButtonStyle())

Button(button_style='warning', description='Add to Train', style=ButtonStyle())

In [34]:
trainmails

,Subject,Body,Recipient,expected classification,category
0,Stoermeldung SM_WERK_SW_10.01.2025_02 aufgetreten,Fehlerklasse: 3\nPLT-SYSTEM: LE <> LE auf Lage...,dummy-inbox@organization.com,important,unimportant\n
1,Neuer Kommentar zu SM_WERK_VO_14.01.2025_01,Name der Stoerungsmeldung: SM_WERK_VO_14.01.20...,dummy-inbox@organization.com,unimportant,unimportant\n
2,Projektupdate: Fortschritt in Sprint 5,"Hallo Team,\n\nDas Projekt Sprint 5 hat einen ...",dummy-inbox@organization.com,unimportant,unimportant\n
3,[External] Incident assigned to your group: IN...,An incident has been assigned to your group.\n...,dummy-inbox@organization.com,important,important\n
4,Projektupdate: Fortschritt in Sprint 5,"Hallo Team,\n\nDas Projekt Sprint 5 hat einen ...",dummy-inbox@organization.com,unimportant,unimportant\n
...,...,...,...,...,...
489,[TicketBCS#288646] Request to update alternati...,"Dear David,\n\nPlease update the alternative b...",dummy-inbox@organization.com,unimportant,unimportant\n
490,Mitarbeiterfeedback benoetigt: Rueckmeldung bi...,"Hallo Team,\n\nWir moechten Sie daran erinnern...",dummy-inbox@organization.com,unimportant,unimportant\n
491,Tickets faellig zur Weiterbearbeitung,"Hallo Joni,\n\nFolgende Tickets sind faellig z...",dummy-inbox@organization.com,important,important\n
492,NaN,,NaN,NaN,NaN
